In [21]:
import dask
import dask.dataframe as dd
import dask.array as da
import numpy as np
import pandas as pd
import ipyparallel as ipp
import functools

dask.set_options(get=dask.multiprocessing.get)



In [22]:
class Groupby:
    def __init__(self, keys):
        _, self.keys_as_int = np.unique(keys, return_inverse = True)
        self.n_keys = max(self.keys_as_int)
        self.set_indices()
        
    def set_indices(self):
        self.indices = [[] for i in range(self.n_keys+1)]
        for i, k in enumerate(self.keys_as_int):
            self.indices[k].append(i)
        self.indices = [np.array(elt) for elt in self.indices]
        
    def apply(self, function, vector, broadcast):
        if broadcast:
            result = np.zeros(len(vector))
            for idx in self.indices:
                result[idx] = function(vector[idx])
        else:
            result = np.zeros(self.n_keys)
            for k, idx in enumerate(self.indices):
                result[self.keys_as_int[k]] = function(vector[idx])

        return result

In [23]:
test_df = pd.DataFrame([[int(ii/100), ii]*10 for ii in range(100000000)])


In [24]:
%%time
test_dask = dd.from_pandas(test_df, npartitions=10)
task = test_dask.groupby(0).apply(np.mean, meta=test_dask)
dask_results = task.compute()

CPU times: user 1min 48s, sys: 4min 1s, total: 5min 50s
Wall time: 5min 47s


In [ ]:
dask_results.shape

(1000000, 20)

In [ ]:
%%time
grp_obj = Groupby(test_df[0])
new_grp = test_df.values[grp_obj.indices]
new_da = da.from_array(new_grp, chunks=(100000,100,20))
task = da.apply_along_axis(np.mean, arr=new_da, axis=1)
task = da.apply_along_axis(np.mean, arr=task, axis=1)
dask_arr_result = task.compute()

In [ ]:
%%time
grp_obj = Groupby(test_df[0])
new_grp = test_df.values[grp_obj.indices]
new_da = da.from_array(new_grp, chunks=(1000,100,20))

task = new_da.mean().compute()

In [ ]:
dask_arr_result.shape

In [ ]:

rc = ipp.Client()
dview = rc[:]
grp = test_df.groupby(0)


In [ ]:
%%time
pd_results = grp.apply(np.mean)

In [ ]:
%%time
def 

grp_obj = Groupby(test_df[0])
new_grp = test_df.values[grp_obj.indices]

ipp_results = dview.map_sync(functools.partial(np.mean, axis=0), new_grp)

In [ ]:
ipp_results[0].shape